This notebook is intended to show the process for going from 
a database connection to two files the first associating an issue
to its components and the second to assocating an issue to its 
comments. These files can then be used a machine learning pipeline
that will apply cleaning, vectorization of the text and building models.

In [ ]:
%pylab inline

In [ ]:
from __future__ import print_function
from __future__ import division

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
import pickle
import MySQLdb as mdb

In [ ]:
connection = mdb.connect(host='', user='', db='monorail')
cursor = connection.cursor()

In [ ]:
def table_to_dataframe(name, connection):
    return pd.read_sql("SELECT * FROM {};".format(name) , con=connection)

In [ ]:
issue = table_to_dataframe('Issue', connection)
comment = table_to_dataframe('Comment', connection)
issue_component = table_to_dataframe('Issue2Component', connection)

In [ ]:
issue.rename(columns={'id':'issue_id'}, inplace=True)
chrome_issue = issue[issue['project_id'] == 16].copy()
chrome_issue_id_set = set(chrome_issue['issue_id'])

### Associate an issue withs its components

In [ ]:
components_by_issue = defaultdict(list)
i = 0
for index, row in issue_component.iterrows():
    if row['issue_id'] in chrome_issue_id_set:
        components_by_issue[row['issue_id']].append(row['component_id'])
    if i % 100000 == 0:
        print(i)
    i += 1

chrome_issue['components'] = chrome_issue['issue_id'].apply(lambda i_id: components_by_issue[i_id])

### Associate an issue withs its comments

In [ ]:
comments_by_issue = defaultdict(list)
i = 0
for index, row in chrome_comment.iterrows():
    comments_by_issue[row["issue_id"]].append((index, row.created))
    if i % 1000000 == 0:
        print(i)
    i += 1

chrome_issue["comments"] = chrome_issue["issue_id"].apply(lambda i_id: 
                                                          [tup[0] for tup 
                                                           in sorted(comments_by_issue[i_id], 
                                                                     key=lambda x: x[1])])

### Only work with closed issues for training

In [ ]:
closed_chrome_issues = chrome_issue[chrome_issue["closed"] > 0]

### Subsample the data (faster to run experiments)

In [ ]:
num_issues = len(closed_chrome_issues)

In [ ]:
issue_subset = closed_chrome_issues.sample(int(num_issues * 0.05))

### Very light cleaning of text (removing markup).

In [ ]:
comment_index_to_text = defaultdict(unicode)

i = 0
for index, row in issue_subset.iterrows():
    for num, comment_id in enumerate(row['comments']):
        text =  BeautifulSoup(comment.loc[comment_id]['content']).get_text().strip().lower()
        comment_index_to_text[comment_id] = text
    
    if i % 10000 == 0:
        print(i)
    i += 1

In [ ]:
issue_subset.to_pickle('subset_issue.pkl')

In [ ]:
pickle.dump(comment_index_to_text, open('comment_text.pkl', 'w'))